# Observational spatial analysis


In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import xarray as xr

from unseen import fileio, time_utils, eva
from acs_plotting_maps import cmap_dict, tick_dict  # NOQA

import spatial_plots

In [ ]:
# Optional parameters
# (This cell is tagged "parameters")
dpi = 300
shapefile = None
shape_overlap = 0.1
alpha = 0.05
time_dim = "time"
lat_dim = "lat"
lon_dim = "lon"
similarity_test = "ks"
gev_relative_fit_test = "bic"
time_agg = "maximum"

In [ ]:
# Required parameters
kwargs = locals()
assert "metric" in kwargs, "Must provide a metric name"
assert "var" in kwargs, "Must provide a variable name"
assert "obs_name" in kwargs, "Must provide a name"

if isinstance(reference_time_period, str):
    reference_time_period = list(reference_time_period.split(" "))

assert os.path.isfile(
    obs_file
), f"Must provide an observations data file (papermill option -p obs_file [filepath])"
assert os.path.isfile(
    gev_params_nonstationary_file
), f"Must provide a nonstationary GEV parameters file (papermill option -p gev_params_nonstationary_file [filepath])"

assert (
    "covariate_base" in kwargs
), "Must provide a nonstationary GEV covariate base year"
assert (
    time_agg in spatial_plots.func_dict
), f"Invalid time aggregation method: {time_agg} (options: {list(spatial_plots.func_dict.keys())})"

# Format parameters passed as strings
assert (
    "gev_trend_period" in kwargs
), "Must provide a GEV trend period (e.g., '[1981, 2010]')"
if isinstance(gev_trend_period, str):
    # Convert string to list
    gev_trend_period = eval(gev_trend_period)
    print(f"gev_trend_period: {gev_trend_period}")

assert (
    "plot_dict" in kwargs
), "Must provide spatial plot dictionary of labels, ticks and colormaps"
if isinstance(plot_dict, str):
    # Convert string to dictionary and check for required keys
    plot_dict = eval(plot_dict)
    for key in [
        "metric",
        "var",
        "var_name",
        "units",
        "units_label",
        "freq",
        "obs_name",
        "cmap",
        "cmap_anom",
        "ticks",
        "ticks_anom",
        "ticks_param_trend",
    ]:
        assert key in plot_dict, f"Missing key: {key} in plot_dict"

### Open dataset of metric in observational data

In [ ]:
obs_ds = fileio.open_dataset(obs_file, shapefile=shapefile, shape_overlap=shape_overlap)
obs_ds

In [ ]:
# Select reference time period (defined in metric config file)
if reference_time_period is not None:
    obs_ds = time_utils.select_time_period(obs_ds, reference_time_period)
obs_ds = obs_ds.dropna("time", how="all")
obs_ds

In [ ]:
# Convert event time strings to cftime objects
event_times = np.vectorize(time_utils.str_to_cftime)(
    obs_ds.event_time, obs_ds.time.dt.calendar
)
obs_ds["event_time"] = (obs_ds.event_time.dims, event_times)
obs_ds

## Spatial Maps

In [ ]:
# Store plot related variables using the InfoSet class
info = spatial_plots.InfoSet(
    name=obs_name, fig_dir=fig_dir, file=obs_file, ds=obs_ds, ds_obs=obs_ds, **plot_dict
)

# Year when max/min event occured

In [ ]:
spatial_plots.plot_event_year(info, obs_ds, time_agg)

## Most common month for max/min event

In [ ]:
spatial_plots.plot_event_month_mode(info, obs_ds)


## Map of metric median

In [ ]:
spatial_plots.plot_time_agg(info, obs_ds, "median")

# Map of metric maximum or minimum

In [ ]:
spatial_plots.plot_time_agg(info, obs_ds, time_agg)

# GEV analysis

In [ ]:
# Load GEV parameters
covariate = obs_ds[time_dim].dt.year
times = xr.DataArray(gev_trend_period, dims="time")

In [ ]:
dparams_ns = fileio.open_dataset(
    gev_params_nonstationary_file, shapefile=shapefile, shape_overlap=shape_overlap
)[var]

dparams_ns

# GEV parameters

In [ ]:
eva.spatial_plot_gev_parameters(
    dparams_ns,
    dataset_name=obs_name,
    outfile=f"{info.fig_dir}/gev_parameters_{info.filestem()}.png",
)

In [ ]:
spatial_plots.plot_gev_param_trend(info, dparams_ns, param="location")

In [ ]:
spatial_plots.plot_gev_param_trend(info, dparams_ns, param="scale")

## Annual reccurence of observed max/min event

In [ ]:
spatial_plots.plot_obs_ari(
    info,
    obs_ds,
    None,
    dparams_ns,
    covariate=covariate_base,
    time_agg=time_agg,
)

## GEV parameter trends

In [ ]:
spatial_plots.plot_gev_param_trend(info, obs_ds, dparams_ns, param="location")

In [ ]:
spatial_plots.plot_gev_param_trend(info, obs_ds, dparams_ns, param="scale")

## Annual exceedance probability 
### GEV-based exceedance probability

In [ ]:
for aep in aep_list:
    spatial_plots.plot_map_aep(
        info,
        obs_ds,
        dparams_ns,
        times,
        aep=aep,
    )

## Probability of breaking the observed record

In [ ]:
spatial_plots.plot_new_record_probability(
    info,
    obs_ds,
    dparams_ns,
    covariate_base,
    time_agg,
    ari=10,
)